In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pandasql
import csv
import os

In [2]:
filename = "../data/weather-underground.csv"
underground = pd.read_csv(filename)

In [3]:
underground.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 70 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                30 non-null     object 
 1   maxpressurem                        30 non-null     int64  
 2   maxdewptm                           30 non-null     int64  
 3   maxpressurei                        30 non-null     float64
 4   maxdewpti                           30 non-null     int64  
 5   since1julheatingdegreedaysnormal    30 non-null     int64  
 6   heatingdegreedaysnormal             30 non-null     int64  
 7   since1sepcoolingdegreedaysnormal    0 non-null      float64
 8   hail                                30 non-null     int64  
 9   since1julsnowfallm                  30 non-null     float64
 10  since1julheatingdegreedays          30 non-null     int64  
 11  maxvisi                             30 non-null

In [4]:
underground.head()

,date,maxpressurem,maxdewptm,maxpressurei,maxdewpti,since1julheatingdegreedaysnormal,heatingdegreedaysnormal,since1sepcoolingdegreedaysnormal,hail,since1julsnowfallm,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
0,2011-05-01,1026,6,30.31,42,4646,8,NaN,0,157.23,...,0.0,0.0,13,0.0,0.0,0,5,60,16,16
1,2011-05-02,1026,10,30.31,50,4653,7,NaN,0,157.23,...,0.0,0.0,14,0.0,0.0,0,13,57,16,14
2,2011-05-03,1021,15,30.14,59,4660,7,NaN,0,157.23,...,0.0,0.0,15,0.0,0.0,0,13,65,16,18
3,2011-05-04,1017,14,30.03,57,4667,7,NaN,0,157.23,...,0.5,0.0,16,12.7,0.0,0,23,55,16,13
4,2011-05-05,1016,4,30.01,39,4673,6,NaN,0,157.23,...,0.0,0.0,17,0.0,0.0,0,32,56,16,13


# Part 1 - Number of rainy days

In [5]:
def num_rainy_days(df):
    '''
    This function should run a SQL query on a dataframe of
    weather data.  The SQL query should return one column and
    one row - a count of the number of days in the dataframe where
    the rain column is equal to 1 (i.e., the number of days it
    rained). 
    '''
    q = """
    SELECT
        COUNT(*)
    FROM
        df
    WHERE
        rain == 1
    """
    
    # Execute the SQL command against the pandas frame
    rainy_days = pandasql.sqldf(q, locals())
    return rainy_days


num_rainy_days(underground)

,COUNT(*)
0,10


# Part 2 - Temp on Foggy and Nonfoggy Days

In [6]:
def max_temp_aggregate_by_fog(df):
    '''
    This function should run a SQL query on a dataframe of
    weather data.  The SQL query should return two columns and
    two rows - whether it was foggy or not (0 or 1) and the max
    maxtempi for that fog value (i.e., the maximum max temperature
    for both foggy and non-foggy days).  The dataframe will be 
    titled 'weather_data'. You'll need to provide the SQL query.
    '''
    q = """
    SELECT
        fog, MAX(maxtempi)
    FROM
        df
    GROUP BY
        fog
    """
    
    # Execute the SQL command against the pandas frame
    foggy_days = pandasql.sqldf(q, locals())
    return foggy_days


max_temp_aggregate_by_fog(underground)

,fog,MAX(maxtempi)
0,0,86
1,1,81


# Part 3 - Mean Temp on Weekends

In [7]:
def avg_weekend_temperature(df):
    '''
    This function should run a SQL query on a dataframe of
    weather data.  The SQL query should return one column and
    one row - the average meantempi on days that are a Saturday
    or Sunday (i.e., the the average mean temperature on weekends).
    
    We can convert dates to days of the week via the 'strftime' keyword in SQL.
    For example, CAST(strftime('%w', date) as integer) will return 0 if the date
    is a Sunday or 6 if the date is a Saturday.
    '''
    q = """
    SELECT
        AVG(meantempi)
    FROM
        df
    WHERE
        CAST(strftime('%w', date) as integer) == 0 OR CAST(strftime('%w', date) as integer) == 6
    """
    
    #Execute the SQL command against the pandas frame
    mean_temp_weekends = pandasql.sqldf(q, locals())
    return mean_temp_weekends


avg_weekend_temperature(underground)

,AVG(meantempi)
0,65.111111


# Part 4 - Mean Temp on Rainy Days

In [8]:
def avg_min_temperature(df):
    '''
    This function should run a SQL query on a dataframe of
    weather data. More specifically you want to find the average
    minimum temperature (mintempi column of the weather dataframe) on 
    rainy days where the minimum temperature is greater than 55 degrees.
    
    You might also find that interpreting numbers as integers or floats may not
    work initially.  In order to get around this issue, it may be useful to cast
    these numbers as integers.  This can be done by writing cast(column as integer).
    So for example, if we wanted to cast the maxtempi column as an integer, we would actually
    write something like where cast(maxtempi as integer) = 76, as opposed to simply 
    where maxtempi = 76.
    '''
    q = """
    SELECT 
        AVG(mintempi)
    FROM
        df
    WHERE
        rain == 1 AND mintempi > 55
    """
    
    #Execute the SQL command against the pandas frame
    avg_min_temp_rainy = pandasql.sqldf(q, locals())
    return avg_min_temp_rainy


avg_min_temperature(underground)

,AVG(mintempi)
0,61.25


# Part 5 - Fixing Turnstile Data

In [9]:
def fix_turnstile_data(filenames):
    '''
    Filenames is a list of MTA Subway turnstile text files. A link to an example
    MTA Subway turnstile text file can be seen at the URL below:
    http://web.mta.info/developers/data/nyct/turnstile/turnstile_110507.txt
    
    As you can see, there are numerous data points included in each row of the
    a MTA Subway turnstile text file. 

    You want to write a function that will update each row in the text
    file so there is only one entry per row. A few examples below:
    A002,R051,02-00-00,05-28-11,00:00:00,REGULAR,003178521,001100739
    A002,R051,02-00-00,05-28-11,04:00:00,REGULAR,003178541,001100746
    A002,R051,02-00-00,05-28-11,08:00:00,REGULAR,003178559,001100775
    
    Write the updates to a different text file in the format of "updated_" + filename.
    For example:
        1) if you read in a text file called "turnstile_110521.txt"
        2) you should write the updated data to "updated_turnstile_110521.txt"

    The order of the fields should be preserved. Remember to read through the 
    Instructor Notes below for more details on the task. 
    
    In addition, here is a CSV reader/writer introductory tutorial:
    http://goo.gl/HBbvyy
    
    You can see a sample of the turnstile text file that's passed into this function
    and the the corresponding updated file by downloading these files from the resources:
    
    Sample input file: turnstile_110528.txt
    Sample updated file: solution_turnstile_110528.txt
    '''
    for name in filenames:
        with open(name, newline='') as csvfile:
            reader = csv.reader(csvfile, skipinitialspace=True)
            rows = [row for row in reader]
        
        head, tail = os.path.split(name)
        out_filename = os.path.join(head, 'updated_' + tail)
        
        with open(out_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for row in rows:
                header = row[: 3]
                row[-1] = row[-1].strip()
                for i in range(0, len(row) - 3, 5):
                    writer.writerow(header + row[3 + i: 3 + i + 5])

                    
files = ["../data/turnstile_110507.txt"]
fix_turnstile_data(files)


with open("../data/updated_turnstile_110507.txt") as csvfile:
    reader = csv.reader(csvfile)
    for _ in range(5):
        print(next(reader))

['A002', 'R051', '02-00-00', '04-30-11', '00:00:00', 'REGULAR', '003143506', '001087907']
['A002', 'R051', '02-00-00', '04-30-11', '04:00:00', 'REGULAR', '003143547', '001087915']
['A002', 'R051', '02-00-00', '04-30-11', '08:00:00', 'REGULAR', '003143563', '001087935']
['A002', 'R051', '02-00-00', '04-30-11', '12:00:00', 'REGULAR', '003143646', '001088024']
['A002', 'R051', '02-00-00', '04-30-11', '16:00:00', 'REGULAR', '003143865', '001088083']


# Part 6 - Combining Turnstile Data

In [10]:
def create_master_turnstile_file(filenames, output_file):
    '''
    Write a function that takes the files in the list filenames, which all have the 
    columns 'C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn', and consolidates
    them into one file located at output_file.  There should be ONE row with the column
    headers, located at the top of the file. The input files do not have column header
    rows of their own.
    
    For example, if file_1 has:
    line 1 ...
    line 2 ...
    
    and another file, file_2 has:
    line 3 ...
    line 4 ...
    line 5 ...
    
    We need to combine file_1 and file_2 into a master_file like below:
     'C/A, UNIT, SCP, DATEn, TIMEn, DESCn, ENTRIESn, EXITSn'
    line 1 ...
    line 2 ...
    line 3 ...
    line 4 ...
    line 5 ...
    '''
    with open(output_file, 'w') as master_file:
        master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename, 'r') as f:
                rows = f.readlines()
                for row in rows:
                    master_file.write(row + '\n')

# Part 7 - Filtering Irregular Data

In [11]:
def filter_by_regular(filename):
    '''
    This function should read the csv file located at filename into a pandas dataframe,
    and filter the dataframe to only rows where the 'DESCn' column has the value 'REGULAR'.
    
    For example, if the pandas dataframe is as follows:
    ,C/A, UNIT,  SCP,    DATEn,   TIMEn,   DESCn, ENTRIESn, EXITSn
    0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
    1,A002,R051,02-00-00,05-01-11,04:00:00,DOOR,3144335,1088159
    2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177
    3,A002,R051,02-00-00,05-01-11,12:00:00,DOOR,3144424,1088231
    
    The dataframe will look like below after filtering to only rows where DESCn column
    has the value 'REGULAR':
    0,A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,3144312,1088151
    2,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177
    '''
    turnstile_data = pd.read_csv(filename)
    
    return turnstile_data[turnstile_data.DESCn == 'REGULAR']


# Part 8 - Get Hourly Entries

In [12]:
def get_hourly_entries(df):
    '''
    The data in the MTA Subway Turnstile data reports on the cumulative
    number of entries and exits per row.  Assume that you have a dataframe
    called df that contains only the rows for a particular turnstile machine
    (i.e., unique SCP, C/A, and UNIT).  This function should change
    these cumulative entry numbers to a count of entries since the last reading
    (i.e., entries since the last row in the dataframe).
    
    More specifically, you want to do two things:
       1) Create a new column called ENTRIESn_hourly
       2) Assign to the column the difference between ENTRIESn of the current row 
          and the previous row. If there is any NaN, fill/replace it with 1.
    
    Examples of what your dataframe should look like at the end of this exercise:
    
           C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn    EXITSn  ENTRIESn_hourly
    0     A002  R051  02-00-00  05-01-11  00:00:00  REGULAR   3144312   1088151                1
    1     A002  R051  02-00-00  05-01-11  04:00:00  REGULAR   3144335   1088159               23
    2     A002  R051  02-00-00  05-01-11  08:00:00  REGULAR   3144353   1088177               18
    3     A002  R051  02-00-00  05-01-11  12:00:00  REGULAR   3144424   1088231               71
    4     A002  R051  02-00-00  05-01-11  16:00:00  REGULAR   3144594   1088275              170
    5     A002  R051  02-00-00  05-01-11  20:00:00  REGULAR   3144808   1088317              214
    6     A002  R051  02-00-00  05-02-11  00:00:00  REGULAR   3144895   1088328               87
    7     A002  R051  02-00-00  05-02-11  04:00:00  REGULAR   3144905   1088331               10
    8     A002  R051  02-00-00  05-02-11  08:00:00  REGULAR   3144941   1088420               36
    9     A002  R051  02-00-00  05-02-11  12:00:00  REGULAR   3145094   1088753              153
    10    A002  R051  02-00-00  05-02-11  16:00:00  REGULAR   3145337   1088823              243
    ...
    ...

    '''
    df['ENTRIESn_hourly'] = df['ENTRIESn'].diff().fillna(1)

    return df


# Part 9 - Get Hourly Exits

In [ ]:
def get_hourly_exits(df):
    '''
    The data in the MTA Subway Turnstile data reports on the cumulative
    number of entries and exits per row.  Assume that you have a dataframe
    called df that contains only the rows for a particular turnstile machine
    (i.e., unique SCP, C/A, and UNIT).  This function should change
    these cumulative exit numbers to a count of exits since the last reading
    (i.e., exits since the last row in the dataframe).
    
    More specifically, you want to do two things:
       1) Create a new column called EXITSn_hourly
       2) Assign to the column the difference between EXITSn of the current row 
          and the previous row. If there is any NaN, fill/replace it with 0.
    
    Example dataframe below:

          Unnamed: 0   C/A  UNIT       SCP     DATEn     TIMEn    DESCn  ENTRIESn    EXITSn  ENTRIESn_hourly  EXITSn_hourly
    0              0  A002  R051  02-00-00  05-01-11  00:00:00  REGULAR   3144312   1088151                0              0
    1              1  A002  R051  02-00-00  05-01-11  04:00:00  REGULAR   3144335   1088159               23              8
    2              2  A002  R051  02-00-00  05-01-11  08:00:00  REGULAR   3144353   1088177               18             18
    3              3  A002  R051  02-00-00  05-01-11  12:00:00  REGULAR   3144424   1088231               71             54
    4              4  A002  R051  02-00-00  05-01-11  16:00:00  REGULAR   3144594   1088275              170             44
    5              5  A002  R051  02-00-00  05-01-11  20:00:00  REGULAR   3144808   1088317              214             42
    6              6  A002  R051  02-00-00  05-02-11  00:00:00  REGULAR   3144895   1088328               87             11
    7              7  A002  R051  02-00-00  05-02-11  04:00:00  REGULAR   3144905   1088331               10              3
    8              8  A002  R051  02-00-00  05-02-11  08:00:00  REGULAR   3144941   1088420               36             89
    9              9  A002  R051  02-00-00  05-02-11  12:00:00  REGULAR   3145094   1088753              153            333
    '''
    
    df['EXITSn_hourly'] = df['EXITSn'].diff().fillna(0)
    return df


# Part 10 - Time to Hour

In [13]:
def time_to_hour(time):
    '''
    Given an input variable time that represents time in the format of:
    "00:00:00" (hour:minutes:seconds)
    
    Write a function to extract the hour part from the input variable time
    and return it as an integer. For example:
        1) if hour is 00, your code should return 0
        2) if hour is 01, your code should return 1
        3) if hour is 21, your code should return 21
        
    Please return hour as an integer.
    '''
    
    hour = int(time.split(':')[0])
    return hour


time = "00:01:30"
time_to_hour(time)

0

# Part 11 - Reformat Subway Dates

In [22]:
import datetime


def reformat_subway_dates(date):
    '''
    The dates in our subway data are formatted in the format month-day-year.
    The dates in our weather underground data are formatted year-month-day.
    
    In order to join these two data sets together, we'll want the dates formatted
    the same way.  Write a function that takes as its input a date in the MTA Subway
    data format, and returns a date in the weather underground format.
    
    Hint: 
    There are a couple of useful functions in the datetime library that will
    help on this assignment, called strptime and strftime. 
    '''

    date = datetime.datetime.strptime(date, "%m-%d-%y")
    return date.strftime("%Y-%m-%d")


date = "12-31-00"
reformat_subway_dates(date)

'2000-12-31'